In [7]:
import pandas as pd
from transformers import pipeline
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud

# Charger les données
df = pd.read_csv('../raw_data/data_cleaned_2021.csv')

# Assurez-vous que la colonne Job Description existe
if 'Job Description' in df.columns:
    job_descriptions = df['Job Description'].dropna().tolist()
else:
    print("La colonne 'Job Description' est absente du fichier.")

In [11]:
# Charger le modèle de Hugging Face pour la reconnaissance d'entités nommées
ner_model = pipeline("ner", grouped_entities=True)

# Fonction pour extraire les compétences
def extract_skills(text, model):
    entities = model(text)
    skills = [entity['word'] for entity in entities if entity['entity_group'] == 'MISC']  # Ajuster selon les étiquettes du modèle
    return skills

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [2]:
# Appliquer l'extraction des compétences à toutes les descriptions de poste
all_skills = []
for desc in job_descriptions:
    skills = extract_skills(desc, ner_model)
    all_skills.extend(skills)

# Compter la fréquence des compétences
skill_counts = Counter(all_skills)

# Créer un nuage de mots pour les compétences
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(skill_counts)

# Afficher le nuage de mots
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nuage de mots des compétences extraites')
plt.show()